# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 30 2022 2:40PM,245369,19,ACG-2102491125,ACG North America LLC,Released
1,Aug 30 2022 2:33PM,245367,10,8463071,Yusen – 3D Matrix,Released
2,Aug 30 2022 2:33PM,245366,10,Eme-103812,Emerginnova,Released
3,Aug 30 2022 2:30PM,245365,10,SO94366,"Senseonics, Incorporated",Released
4,Aug 30 2022 2:26PM,245364,10,0085953802,ISDIN Corporation,Released
5,Aug 30 2022 2:26PM,245364,10,0085953816,ISDIN Corporation,Released
6,Aug 30 2022 2:26PM,245364,10,0085953824,ISDIN Corporation,Released
7,Aug 30 2022 2:26PM,245364,10,0085953825,ISDIN Corporation,Released
8,Aug 30 2022 2:26PM,245364,10,0085953827,ISDIN Corporation,Released
9,Aug 30 2022 2:26PM,245364,10,0085953826,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,245364,Released,40
44,245365,Released,1
45,245366,Released,1
46,245367,Released,1
47,245369,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245364,NaN,NaN,40.0
245365,NaN,NaN,1.0
245366,NaN,NaN,1.0
245367,NaN,NaN,1.0
245369,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245240,1.0,7.0,30.0
245259,26.0,35.0,3.0
245276,0.0,0.0,1.0
245277,0.0,0.0,1.0
245280,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245240,1,7,30
245259,26,35,3
245276,0,0,1
245277,0,0,1
245280,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245240,1,7,30
1,245259,26,35,3
2,245276,0,0,1
3,245277,0,0,1
4,245280,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245240,1,7,30
1,245259,26,35,3
2,245276,,,1
3,245277,,,1
4,245280,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 30 2022 2:40PM,245369,19,ACG North America LLC
1,Aug 30 2022 2:33PM,245367,10,Yusen – 3D Matrix
2,Aug 30 2022 2:33PM,245366,10,Emerginnova
3,Aug 30 2022 2:30PM,245365,10,"Senseonics, Incorporated"
4,Aug 30 2022 2:26PM,245364,10,ISDIN Corporation
44,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.
45,Aug 30 2022 1:43PM,245360,15,"Virtus Pharmaceuticals, LLC"
52,Aug 30 2022 1:42PM,245340,15,"Chadwick Pharmaceuticals, Inc."
53,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd"
82,Aug 30 2022 1:39PM,245359,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 30 2022 2:40PM,245369,19,ACG North America LLC,,,1
1,Aug 30 2022 2:33PM,245367,10,Yusen – 3D Matrix,,,1
2,Aug 30 2022 2:33PM,245366,10,Emerginnova,,,1
3,Aug 30 2022 2:30PM,245365,10,"Senseonics, Incorporated",,,1
4,Aug 30 2022 2:26PM,245364,10,ISDIN Corporation,,,40
5,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,,,1
6,Aug 30 2022 1:43PM,245360,15,"Virtus Pharmaceuticals, LLC",,,7
7,Aug 30 2022 1:42PM,245340,15,"Chadwick Pharmaceuticals, Inc.",,,1
8,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd",,9,20
9,Aug 30 2022 1:39PM,245359,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 2:40PM,245369,19,ACG North America LLC,1,,
1,Aug 30 2022 2:33PM,245367,10,Yusen – 3D Matrix,1,,
2,Aug 30 2022 2:33PM,245366,10,Emerginnova,1,,
3,Aug 30 2022 2:30PM,245365,10,"Senseonics, Incorporated",1,,
4,Aug 30 2022 2:26PM,245364,10,ISDIN Corporation,40,,
5,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,1,,
6,Aug 30 2022 1:43PM,245360,15,"Virtus Pharmaceuticals, LLC",7,,
7,Aug 30 2022 1:42PM,245340,15,"Chadwick Pharmaceuticals, Inc.",1,,
8,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd",20,9,
9,Aug 30 2022 1:39PM,245359,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 2:40PM,245369,19,ACG North America LLC,1,,
1,Aug 30 2022 2:33PM,245367,10,Yusen – 3D Matrix,1,,
2,Aug 30 2022 2:33PM,245366,10,Emerginnova,1,,
3,Aug 30 2022 2:30PM,245365,10,"Senseonics, Incorporated",1,,
4,Aug 30 2022 2:26PM,245364,10,ISDIN Corporation,40,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 2:40PM,245369,19,ACG North America LLC,1.0,NaN,NaN
1,Aug 30 2022 2:33PM,245367,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Aug 30 2022 2:33PM,245366,10,Emerginnova,1.0,NaN,NaN
3,Aug 30 2022 2:30PM,245365,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Aug 30 2022 2:26PM,245364,10,ISDIN Corporation,40.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 2:40PM,245369,19,ACG North America LLC,1.0,0.0,0.0
1,Aug 30 2022 2:33PM,245367,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Aug 30 2022 2:33PM,245366,10,Emerginnova,1.0,0.0,0.0
3,Aug 30 2022 2:30PM,245365,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Aug 30 2022 2:26PM,245364,10,ISDIN Corporation,40.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3189360,58.0,19.0,11.0
15,981295,23.0,35.0,60.0
19,981427,22.0,10.0,0.0
20,1226549,105.0,30.0,1.0
21,2207547,8.0,1.0,0.0
22,245347,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3189360,58.0,19.0,11.0
1,15,981295,23.0,35.0,60.0
2,19,981427,22.0,10.0,0.0
3,20,1226549,105.0,30.0,1.0
4,21,2207547,8.0,1.0,0.0
5,22,245347,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,58.0,19.0,11.0
1,15,23.0,35.0,60.0
2,19,22.0,10.0,0.0
3,20,105.0,30.0,1.0
4,21,8.0,1.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,58.0
1,15,Released,23.0
2,19,Released,22.0
3,20,Released,105.0
4,21,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,11.0,60.0,0.0,1.0,0.0,0.0
Executing,19.0,35.0,10.0,30.0,1.0,0.0
Released,58.0,23.0,22.0,105.0,8.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,11.0,60.0,0.0,1.0,0.0,0.0
1,Executing,19.0,35.0,10.0,30.0,1.0,0.0
2,Released,58.0,23.0,22.0,105.0,8.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,11.0,60.0,0.0,1.0,0.0,0.0
1,Executing,19.0,35.0,10.0,30.0,1.0,0.0
2,Released,58.0,23.0,22.0,105.0,8.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()